# Preliminaries


[Catalog of sources](https://cdsarc.unistra.fr/viz-bin/cat/J/A%2bA/628/A66)

Not sure how to render this:
```
@MISC{ vizier:J/A+A/628/A66, 
author = {Lodieu N., Perez-Garrido A., Smart R.L., Silvotti, R.}, 
title = "{VizieR Online Data Catalog: alpha Persei, Pleiades and Praesepe clusters}", 
year = "2019", 
doi = {10.26093/cds/vizier.36280066}, 
journal = {\aap}
}
```

In [1]:
import astropy.coordinates as coord
from astropy.table import QTable, vstack

import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.units import Quantity
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
import pandas as pd
import pickle
import os

Created TAP+ (v1.2.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443
Created TAP+ (v1.2.1) - Connection:
	Host: geadata.esac.esa.int
	Use HTTPS: True
	Port: 443
	SSL Port: 443


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm
import numpy as np

In [3]:
from cluster import cluster as cl
from fieldstars import fieldstars as fs

# Get Lodieu Data Files

Details of processing these data files are [here](https://drive.google.com/file/d/1-BXGwshTcOD1bptRXy_pKGCemQnDY22z/view?usp=sharing)


In [4]:
alpha_per_lodieu = cl('alpha_per','./data/J_A+A_628_A66')

In [7]:
#column mapper to map to field star class
colmapper= {'source_id':'SourceCP', 'ra':'RAdeg', 'dec':'DEdeg','parallax':'Plx',
            'pmra':'pmRA', 'pmdec':'pmDE', 'radial_velocity':'RVCP',
            'phot_g_mean_mag':'Gmag', 'phot_bp_mean_mag':'BPmag', 'phot_rp_mean_mag':'RPmag','r_est':'rest'}

In [8]:
from fieldstars import from_pandas
alpha_per=from_pandas(alpha_per_lodieu.objs, colmapper, name='alpha_per')

In [9]:
alpha_per.name

'alpha_per'

In [11]:
len(alpha_per.objs)

3162

In [15]:
alpha_per.maxsep()

<Angle 11.46247503 deg>

In [18]:
len(alpha_per.coords)

3162

In [12]:
alpha_per_lodieu.coldefs['distC']

{'units': 'pc',
 'desc': 'Distance from the center of the cluster (dist_center)'}

In [13]:
alpha_per_lodieu.coldefs['bx']

{'units': 'pc', 'desc': 'Galactic X coordinate (b_x)'}

In [14]:
alpha_per_lodieu.findcolumns('parallax')

[('PlxCP', 'Parallax (Plx_CP)'),
 ('e_PlxCP', 'Parallax error (ePlx_CP)'),
 ('Plx', 'Gaia DR2 parallax (parallax)'),
 ('e_Plx', 'Gaia DR2 parallax error (parallax_error)'),
 ('RAPlxcor',
  '? Correlation between right ascension and parallax (ra_parallax_corr)'),
 ('DEPlxcor',
  '? Correlation between declination and parallax (dec_parallax_corr)'),
 ('PlxpmRAcor',
  '? Correlation between parallax and proper motion in right ascension (parallax_pmra_corr)'),
 ('PlxpmDEcor',
  '? Correlation between parallax and proper motion in declination (parallax_pmdec_corr)'),
 ('fvarpi', 'Mean Parallax factor AL (mean_varpi_factor_al)')]

### Cut the data by distance from Cluster Center

In [11]:
cuts = np.array([0,2.3,5.6, 9.5, 28.5,50,75,100])
labs = [f'Within {d} pc' for d in cuts[1:]]
#sm = cm.ScalarMappable(cmap=cm.Set1)
#bin_colors = sm.to_rgba(range(len(labs)), alpha=0.8)
bin_colors = ['black', 'blue', 'green', 'red','magenta','orange', 'yellow']
alpha_per.objs['DistanceBin']  =pd.cut(alpha_per.objs.distC, cuts, labels=labs)


AttributeError: 'DataFrame' object has no attribute 'distC'

In [12]:
alpha_per.objs.DistanceBin.value_counts()

AttributeError: 'DataFrame' object has no attribute 'DistanceBin'

# Lodieu Plots

The plots below reproduce Figure A.1 in Appendix A of Lodieu's paper. Not all of the 3,162 objects in the data file are plotted, just the 2,069 objects within 28.5 parsecs of $\alpha Per$ center in keeping with the Lodieu's methodology.

In [13]:
#@title RA, Dec, and PM

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

for i in range(3,-1,-1):
  pdf = alpha_per.objs.query('DistanceBin == @labs[@i]')
  ax[0].scatter(pdf.RAdeg, pdf.DEdeg, color=bin_colors[i], label=labs[i], s=4)
#ax[0].legend(loc='center right', bbox_to_anchor=(1.4, 0.75))
ax[0].set_xlabel('Right Ascension')
_=ax[0].set_ylabel('Declination')

for i in range(3,-1,-1):
  pdf = alpha_per.objs.query('DistanceBin == @labs[@i]')
  ax[1].scatter(pdf.pmRA, pdf.pmDE, color=bin_colors[i], label=labs[i], s=4)
ax[1].legend(loc='center right', bbox_to_anchor=(1.4, 0.75))
ax[1].set_xlabel('PM RA (mas/yr)')
_=ax[1].set_ylabel('PM Dec (mas/yr)')


UndefinedVariableError: name 'DistanceBin' is not defined

In [14]:
#@title Magnitude and PM v. Parallax

fig, ax = plt.subplots(1, 2, figsize=(12, 6))

for i in range(3,-1,-1):
  pdf = alpha_per.objs.query('DistanceBin == @labs[@i]')
  ax[0].scatter(pdf.Plx, pdf.Gmag, color=bin_colors[i], label=labs[i], s=4)
#ax[0].legend(loc='center right', bbox_to_anchor=(1.4, 0.75))
ax[0].set_xlabel('Gaia G-Mag')
ax[0].invert_yaxis()
_=ax[0].set_ylabel('Gaia Parallax (MAS)')

for i in range(3,-1,-1):
  pdf = alpha_per.objs.query('DistanceBin == @labs[@i]')
  ax[1].scatter(pdf.Plx,
                np.sqrt(pdf.pmRA**2 + pdf.pmDE**2),
                 color=bin_colors[i], label=labs[i], s=4)
ax[1].legend(loc='center right', bbox_to_anchor=(1.4, 0.75))
ax[1].set_ylabel('Total PM (MAS/yr')
_=ax[1].set_xlabel('Gaia Parallax (MAS)')

UndefinedVariableError: name 'DistanceBin' is not defined

In [15]:
#@title Distance from Center Histograms
fig, ax = plt.subplots(1, 2, figsize=(16, 6))
zz=ax[0].hist(alpha_per.objs.distC, bins=np.linspace(0,500,100, endpoint=True))
ax[0].set_ylabel('Number of objects')
un = u.parsec
ax[0].set_xlabel(f'Distance ({un})')
ax[0].set_title('Distance From Cluster Center')
N = len(alpha_per.objs)
mode = zz[1][zz[0].argmax()]
ax[0].text(0.60, 0.60,f'N = {N}\nmodal bucket = {mode:.0f} pc',transform=ax[0].transAxes)
ax[0].grid(axis='y')
#ax.axvline(alf_per_dist, color='red',label=r'$\alpha Per$')
#ax.legend()

alpha_per.objs.groupby('DistanceBin').size().plot(kind='bar', ax=ax[1])
ax[1].set_ylabel('Number of Stars')
ax[1].set_xlabel('Distance from Center')
ax[1].grid(axis='y')
ax[1].set_title("Count by Distance Bin")

AttributeError: 'DataFrame' object has no attribute 'distC'

# Query Gaia

In [16]:
alpha_per.findcolumns('gaia')

AttributeError: 'fieldstars' object has no attribute 'findcolumns'

In [17]:
alpha_per.findcolumns('phot_.*_mean_mag')

AttributeError: 'fieldstars' object has no attribute 'findcolumns'

From Lodieu mean distance to $\alpha Per$ is Per $177.68\pm0.84 pc $ so we'll query 170 and 185 PC.

In [18]:
np.median(alpha_per.objs.rest),alpha_per.objs.rest.mean(), alpha_per.objs.rest.std(), alpha_per.objs.rest.max()

AttributeError: 'DataFrame' object has no attribute 'rest'

In [19]:
# center of the observations:
ra = alpha_per.RAdeg.mean(); dec = alpha_per.DEdeg.mean()

#angular distance from center for each obs:
ang_dist = np.sqrt((alpha_per.RAdeg - ra)**2 + (alpha_per.DEdeg-dec)**2)
search_r = ang_dist.max()

#distance constraint (10% on either side of median):
med_dist = np.median(alpha_per.rest)

print(f'Search cone: ra: {ra}, dec: {dec}, search radius: {search_r}')
print(f'Distance constraint: between {med_dist*.9} and {med_dist*1.1}')

AttributeError: 'fieldstars' object has no attribute 'RAdeg'

In [20]:
zz=plt.hist(ang_dist,bins=50)

NameError: name 'ang_dist' is not defined

In [21]:
#@title Tap Query String
tap_query_string = '\n'.join([
'select \tgs.source_id as Source ',
        '\t\t,gs.ra as RAdeg ',
        '\t\t,gs.dec as DEdeg ',
        '\t\t,gs.parallax as Plx ',
        '\t\t,gs.pmra as pmRA ',
        '\t\t,gs.pmdec as pmDE ',
        '\t\t,gs.phot_g_mean_mag as Gmag ',
        '\t\t,gs.phot_bp_mean_mag as BPmag ',
        '\t\t,gs.phot_rp_mean_mag as RPmag ',
        '\t\t,gd.r_est as rest ',
    'from gaiadr2.gaia_source gs ',
    '\tleft join gaiadr2_complements.geometric_distance gd using (source_id) ',
    'where ', 
          '\tCONTAINS(POINT(\'ICRS\', gs.ra, gs.dec), ',
              '\t\tCIRCLE(\'ICRS\', {ra}, {dec}, {rad})) = 1 '.format(ra=ra, dec=dec, rad=search_r),
      '\tand gd.r_est between {r_lo} and {r_hi}'.format(r_lo=med_dist*0.9, r_hi=med_dist*1.1),
      '\tand gs.parallax_over_error > 5.0'
])

print(tap_query_string)

NameError: name 'ra' is not defined

In [22]:
"""
#1. Import the relevant modules/classes/packages
from pyvo.dal import TAPService

#2. Query the ARI Gaia TAP Service
tap_service_url = "http://gaia.ari.uni-heidelberg.de/tap"
tap_service = TAPService(tap_service_url)
tap_results = tap_service.search(tap_query_string);

print(f'query returned {len(tap_results)} rows')
"""

'\n#1. Import the relevant modules/classes/packages\nfrom pyvo.dal import TAPService\n\n#2. Query the ARI Gaia TAP Service\ntap_service_url = "http://gaia.ari.uni-heidelberg.de/tap"\ntap_service = TAPService(tap_service_url)\ntap_results = tap_service.search(tap_query_string);\n\nprint(f\'query returned {len(tap_results)} rows\')\n'

In [23]:
alpha_per.set_index('Source', inplace=True)

AttributeError: 'fieldstars' object has no attribute 'set_index'

In [24]:
import pickle
path = './data/tap_query.pkl'
with open(path, 'rb') as pkl:
    tap_df = pickle.load(pkl)

In [25]:
tap_df = tap_results.to_table().to_pandas()
tap_df.set_index('source', inplace=True)

NameError: name 'tap_results' is not defined

In [26]:
"""
import pickle
path = './data/tap_query.pkl'
with open(path,'wb') as pkl:
  pickle.dump(tap_df, pkl)
"""

"\nimport pickle\npath = './data/tap_query.pkl'\nwith open(path,'wb') as pkl:\n  pickle.dump(tap_df, pkl)\n"

In [27]:
tap_df.head()

,radeg,dedeg,plx,pmra,pmde,gmag,bpmag,rpmag,rest
source,,,,,,,,,
216762910621557504,57.724542,31.839752,5.001295,-2.726881,-30.689681,17.566145,19.199448,16.031351,201.138437
216776968051779200,57.920635,31.922662,5.102463,5.721302,4.014477,13.595032,14.411274,12.714785,194.882810
216810984192641024,57.213933,31.894783,5.843610,23.298598,-41.769166,17.310120,19.057116,16.039265,170.474358
216442162463613184,56.641810,31.709988,5.931883,3.279264,-40.023735,19.815424,20.896929,18.330465,173.087409
216439349261434496,56.850610,31.718682,5.682356,32.390959,-20.722082,17.839262,19.715315,16.479885,175.519125


### Make Consolidated Data Frame

In [28]:
#need the colnames in lower case because that's what the tap query returns
alpha_per_lc = alpha_per.rename(columns = dict([(s,s.lower()) for s in alpha_per.columns]))

AttributeError: 'fieldstars' object has no attribute 'rename'

In [29]:
alpha_per_lc.head()

NameError: name 'alpha_per_lc' is not defined

In [30]:
mycols = [s.lower() for s in ['radeg','DEdeg','Plx', 'pmRA', 'pmDE','Gmag','BPmag','RPmag','rest']]

consol_df=tap_df.merge(alpha_per_lc[mycols],how='outer',left_index=True, right_index=True,indicator=True)

NameError: name 'alpha_per_lc' is not defined

In [31]:
consol_df._merge.value_counts()

NameError: name 'consol_df' is not defined

In [32]:
consol_df['which'] = consol_df._merge.apply(lambda s: 'lodieu' if s == 'right_only' else 'tap' if s == 'left_only' else 'both')
for c in mycols:
  consol_df[c]=np.where(np.isnan(consol_df[c+'_x']), consol_df[c+'_y'], consol_df[c+'_x'])

consol_df.drop(columns = [s+'_x' for s in mycols]+[s+'_y' for s in mycols], inplace=True)

NameError: name 'consol_df' is not defined

In [33]:
consol_df.which.value_counts()

NameError: name 'consol_df' is not defined

In [34]:
consol_df.query('which == \'lodieu\'').head()

NameError: name 'consol_df' is not defined

In [35]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

for s,c in zip(['tap','both','lodieu',],['red','green','blue']):
  pdf = consol_df.query('which == @s')
  ax[0].scatter(pdf.radeg, pdf.dedeg, color=c, label=s, s=2, alpha=0.3)
#ax[0].legend(loc='center right', bbox_to_anchor=(1.4, 0.75))
ax[0].set_xlabel('Right Ascension')
_=ax[0].set_ylabel('Declination')

for s,c in zip(['tap','both','lodieu'],['red','green','blue']):
  pdf = consol_df.query('which == @s')
  ax[1].scatter(pdf.pmra, pdf.pmde, color=c, label=s, s=2, alpha=0.3)
ax[1].legend(loc='center right', bbox_to_anchor=(1.4, 0.75))
ax[1].set_xlabel('PM RA (mas/yr)')
_=ax[1].set_ylabel('PM Dec (mas/yr)')


NameError: name 'consol_df' is not defined

In [37]:
consol_df.query('which == \'lodeiu\'').head()

NameError: name 'consol_df' is not defined

In [38]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
for s,c in zip(['tap','both','lodieu',],['red','green','blue']):
  pdf = consol_df.query('which == @s')
  ax.set_yscale('log')
  ax.hist(pdf.rest,bins=np.linspace(0,500,100, endpoint=True), color=c, label=s,  alpha=0.3)
ax.legend()

NameError: name 'consol_df' is not defined

In [39]:
consol_df.query('rest >= 165.15045  and rest <=201.85055').which.value_counts()

NameError: name 'consol_df' is not defined

In [40]:
#build balanced data frame
members = consol_df.query('which==\'both\'')
n_both = len(members)
non_members = consol_df.query('which==\'tap\'').sample(n=n_both, random_state=1234)
balanced_df = pd.concat([members, non_members])

#shuffle it up
np.random.seed(4321)
balanced_df = balanced_df.loc[np.random.permutation(balanced_df.index)]

#ditch rows with NaNs
balanced_df.dropna(inplace=True)

#binary value for candidacy
balanced_df['candidate'] = np.where(balanced_df.which == 'both', 1, 0)

balanced_df.which.value_counts()

NameError: name 'consol_df' is not defined

In [41]:
balanced_df.info()

NameError: name 'balanced_df' is not defined

In [42]:
np.any(balanced_df.gmag==-9999.0)

NameError: name 'balanced_df' is not defined

In [43]:
# partition to train, val and test dataframes
test_frac = 0.2; train_frac = 0.65; val_frac = 1.0 - test_frac - train_frac
nrows = len(balanced_df)
test_n = int(test_frac*nrows); train_n=int(train_frac*nrows); val_n=nrows-test_n-train_n

train_df = balanced_df.iloc[:train_n]
val_df = balanced_df.iloc[train_n:train_n+val_n]
test_df = balanced_df.iloc[train_n+val_n:]

len(train_df), len(val_df), len(test_df), nrows

NameError: name 'balanced_df' is not defined

# Classifier Models

In [44]:
# some needed libraries
from sklearn.model_selection import train_test_split # to partition the dataset into training and test
from sklearn.metrics import accuracy_score, confusion_matrix # for model evaluation
from sklearn.linear_model import LogisticRegression # classifier model to test

In [45]:
# function to create features from input data set
# returns (n,4) feature matrix and (n,) label vector
def get_features_labels(df):
    features = np.zeros((len(df), 5))
    features[:,0] = np.array(df.pmra)
    features[:,1] = np.array(df.pmde)
    features[:,2] = np.array(df.bpmag - df.gmag)
    features[:,3] = np.array(df.gmag - df.rpmag)
    features[:,4] = np.array(np.sqrt(df.pmra**2+df.pmde**2))
    labels = np.array(df.candidate)
    return features, labels

In [46]:
#get the features and labels for each of the sets
x_train,  y_train  = get_features_labels(train_df)
x_val,  y_val  = get_features_labels(val_df)
x_test, y_test = get_features_labels(test_df)

NameError: name 'train_df' is not defined

## Logit Classifier

In [47]:
logit_model = LogisticRegression()
logit_model.fit(x_train,y_train)

NameError: name 'x_train' is not defined

In [49]:
# calculate prediction accuracy on the dev set
pred_ovr_train = logit_model.predict(x_train)
acc_ovr_train = accuracy_score(y_train, pred_ovr_train)

#calculate prediction accuracy on the validation set
pred_ovr_val = logit_model.predict(x_val)
acc_ovr_val = accuracy_score(y_val, pred_ovr_val)

print('OVR Training Set Accuracy: ', acc_ovr_train)
print('OVR Validation Set Accuracy: ', acc_ovr_val)

NameError: name 'x_train' is not defined

In [50]:
confusion_matrix(y_val, pred_ovr_val)

NameError: name 'y_val' is not defined

In [51]:
logit_model.classes_

AttributeError: 'LogisticRegression' object has no attribute 'classes_'

## K Nearest Neighbors

In [52]:
from sklearn.neighbors import KNeighborsClassifier

In [53]:
neighbors = KNeighborsClassifier()
neighbors.fit(x_train, y_train)

NameError: name 'x_train' is not defined

In [54]:
# calculate prediction accuracy on the dev set
pred_knn_train = neighbors.predict(x_train)
acc_knn_train = accuracy_score(y_train, pred_knn_train)

#calculate prediction accuracy on the validation set
pred_knn_val = neighbors.predict(x_val)
acc_knn_val = accuracy_score(y_val, pred_knn_val)

print('KNN Training Set Accuracy: ', acc_knn_train)
print('KNN Validation Set Accuracy: ', acc_knn_val)

NameError: name 'x_train' is not defined

In [55]:
confusion_matrix(y_val, pred_knn_val)

NameError: name 'y_val' is not defined

## Decision Tree Classifier

In [56]:
from sklearn.tree import DecisionTreeClassifier

In [57]:
dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)

NameError: name 'x_train' is not defined

In [58]:
# calculate prediction accuracy on the dev set
pred_dtc_train = dtc.predict(x_train)
acc_dtc_train = accuracy_score(y_train, pred_dtc_train)

#calculate prediction accuracy on the validation set
pred_dtc_val = dtc.predict(x_val)
acc_dtc_val = accuracy_score(y_val, pred_dtc_val)

print('DTC Training Set Accuracy: ', acc_dtc_train)
print('DTC Validation Set Accuracy: ', acc_dtc_val)

NameError: name 'x_train' is not defined

## Ensemble

In [59]:
def ensemble(modlist, X):
  """
  only works for binary classifiers
  """
  thresh = np.ceil(len(modlist)/2.0)
  yhat = np.array([m.predict(X) for m in modlist])
  yhat_ens = np.where(yhat.sum(axis=0)>=thresh,1,0)
  return yhat_ens


In [60]:
modlist = [logit_model, neighbors, dtc]

In [61]:
# calculate prediction accuracy on the dev set
pred_ens_train = ensemble(modlist, x_train)
acc_ens_train = accuracy_score(y_train, pred_ens_train)

#calculate prediction accuracy on the validation set
pred_ens_val = ensemble(modlist, x_val)
acc_ens_val = accuracy_score(y_val, pred_ens_val)

print('ENS Training Set Accuracy: ', acc_ens_train)
print('ENS Validation Set Accuracy: ', acc_ens_val)

NameError: name 'x_train' is not defined

In [62]:
confusion_matrix(y_val, pred_ens_val)

NameError: name 'y_val' is not defined

In [63]:
confusion = pd.DataFrame([confusion_matrix(y_val, pred_ovr_val).flatten(),
                          confusion_matrix(y_val, pred_knn_val).flatten(),
                          confusion_matrix(y_val, pred_dtc_val).flatten(),
                          confusion_matrix(y_val, pred_ens_val).flatten()],
                    index=['LogitRegr','KNN', 'DTC', 'Ens'],
                    columns = ['Tpos','Fneg','Fpos','Tneg'])

NameError: name 'y_val' is not defined

In [65]:
confusion

NameError: name 'confusion' is not defined

In [66]:
acc_ens_train.shape, pred_dtc_train.shape

NameError: name 'acc_ens_train' is not defined

In [67]:
train_acc = (acc_ovr_train, acc_knn_train, acc_dtc_train, acc_ens_train)
val_acc = (acc_ovr_val, acc_knn_val, acc_dtc_val, acc_ens_val)

ind = np.arange(len(train_acc))  # the x locations for the groups
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(ind - width/2, train_acc, width, 
                label='Training')
rects2 = ax.bar(ind + width/2, val_acc, width, 
                label='Validation')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Scores by Method and Partition')
ax.set_xticks(ind)
ax.set_xticklabels(('LogitRegr', 'KNN', 'DecisionTree', 'Ensemble'))
ax.legend()

NameError: name 'acc_ovr_train' is not defined

# Old Stuff

In [68]:
def get_stars(ra, dec, rad=2.0*u.degree):
    query_text = "SELECT  \
                random_index, ra, dec, parallax, pmra, pmdec, radial_velocity,\
                phot_g_mean_mag, phot_bp_mean_mag, phot_rp_mean_mag, teff_val, radius_val, lum_val \
                FROM gaiadr2.gaia_source \
            WHERE CONTAINS(POINT('ICRS',gaiadr2.gaia_source.ra,gaiadr2.gaia_source.dec), \
                    CIRCLE('ICRS',{ra},{dec},{rad}))=1 \
                    and parallax_over_error > 5 ".format(ra=ra.value, dec=dec.value, rad=rad.value)
                    #--AND abs(pmra_error/pmra)<0.10 \
                    #--AND abs(pmdec_error/pmdec)<0.10 \
                    #--AND pmra IS NOT NULL AND abs(pmra)>0 \
                    #--AND pmdec IS NOT NULL AND abs(pmdec)>0 \
                    #--and parallax IS NOT NULL and parallax > 0".format(ra=ra.value, dec=dec.value, rad=rad.value)
    
    job = Gaia.launch_job_async(query_text)
    gaia_data = job.get_results()

    result_dict = {}
    result_dict['dist'] = coord.Distance(parallax=u.Quantity(gaia_data['parallax']),allow_negative=True)
    result_dict['coord']=  coord.SkyCoord(ra=gaia_data['ra'],
                   dec=gaia_data['dec'],
                   distance=result_dict['dist'],
                   pm_ra_cosdec=gaia_data['pmra'],
                   pm_dec=gaia_data['pmdec'],
                   radial_velocity=gaia_data['radial_velocity'])
    result_dict['M_G'] = gaia_data['phot_g_mean_mag'] - result_dict['dist'].distmod
    result_dict['BP_RP'] = gaia_data['phot_bp_mean_mag'] - gaia_data['phot_rp_mean_mag']
    
    result_dict['data'] = gaia_data.to_pandas()

    return result_dict

In [69]:
#don't execute this query
#clust = get_stars(alf_per.ra, alf_per.dec)

In [70]:
#nor this
#len(clust['data'])

In [71]:
# the file colnames.txt extracted from Readme.txt that came with the archive.  Provides column names for the alpha_per and pleiades data sets
colnames_txt_path = './gdrive/My Drive/cas-tau/Cass-Tau/data/J_A+A_628_A66/colnames.txt'
with open(colnames_txt_path,'r') as f:
    colnames_txt = f.readlines()
colnames = [s.strip() for s in colnames_txt]
colnames = [s for s in colnames if s] #ditch the blanks

FileNotFoundError: [Errno 2] No such file or directory: './gdrive/My Drive/cas-tau/Cass-Tau/data/J_A+A_628_A66/colnames.txt'

In [72]:
# read up the two data files
path = './gdrive/My Drive/cas-tau/Cass-Tau/data/J_A+A_628_A66/alphaper.dat'
alpha_per = pd.read_csv(path, delim_whitespace=True, error_bad_lines=False, warn_bad_lines=True,
                        header=None,usecols=range(len(colnames)), names=colnames)
path = './gdrive/My Drive/cas-tau/Cass-Tau/data/J_A+A_628_A66/pleiades.dat'
pleiades = pd.read_csv(path, delim_whitespace=True, error_bad_lines=False, warn_bad_lines=True,
                        header=None,usecols=range(len(colnames)), names=colnames)

NameError: name 'colnames' is not defined

In [73]:
alpha_per.info()

AttributeError: 'fieldstars' object has no attribute 'info'

In [74]:
(alpha_per.RVCP==-9999.0).sum()/len(alpha_per)

AttributeError: 'fieldstars' object has no attribute 'RVCP'

In [75]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
zz=ax.hist(alpha_per.rest,bins=np.linspace(0,500,50, endpoint=True))
ax.set_ylabel('Number of objects')
un = u.parsec
ax.set_xlabel(f'Distance ({un})')
ax.set_title('Distance Histogram')
N = len(alpha_per)
mode = zz[1][zz[0].argmax()]
ax.text(0.60, 0.60,f'N = {N}\nmodal bucket = {mode:.0f} pc',transform=ax.transAxes)
ax.grid(axis='y')
ax.axvline(alf_per_dist, color='red',label=r'$\alpha Per$')
ax.legend()

AttributeError: 'fieldstars' object has no attribute 'rest'

In [76]:
clust['data'].head()

NameError: name 'clust' is not defined

In [77]:
fig, ax = plt.subplots(1, 2, figsize=(12, 6))

zz=ax[0].hist(np.array(clust['dist']),bins=50, alpha=0.5)
ax[0].set_ylabel('Number of objects')
un = clust['dist'][0].unit
ax[0].set_xlabel(f'Distance ({un})')
ax[0].set_title('Distance Histogram')
N = len(clust['data'])
ax[0].text(6000,4100,f'N = {N}')
ax[0].grid(axis='y')
ax[0].axvline(155, color='red',label=r'$\alpha Per$')
ax[0].legend()


pcm=ax[1].scatter(clust['data'].pmra, clust['data'].pmdec, s=1., alpha=0.5,
           c = clust['dist'],
           cmap='plasma')

un = clust['dist'][0].unit

cbar = fig.colorbar(pcm)
cbar.set_label(f'Distance ({un})', rotation=270)
ax[1].set_xlim(-75,75)
ax[1].set_ylim(-75,75)
ax[1].set_title('Proper Motions')
ax[1].set_ylabel('Declination')
ax[1].set_xlabel('Right Ascension')

NameError: name 'clust' is not defined

In [78]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

pcm = ax.scatter(clust['BP_RP'], clust['M_G'],
        marker='.', alpha=0.5,c = 'blue', s=2) #,
           #cmap='plasma_r', norm=colors.PowerNorm(gamma=0.5,vmin=gaia_data_r['teff_val'].min(), vmax=gaia_data_r['teff_val'].max()))

#ax.set_xlim(0, 3)
#ax.set_ylim(11, 1)
ax.invert_yaxis()

ax.set_xlabel('$G_{BP}-G_{RP}$')
ax.set_ylabel('$M_G$')
ax.set_title('HR Diagram')
#cbar = fig.colorbar(pcm)
#cbar.ax.set_ylabel( 'Effective Temperature (K)', rotation=270)

NameError: name 'clust' is not defined